In [1]:
#!/usr/bin/env python
"""
Script to calculate Mutual Information between two discrete random variables

Roberto Maestre - rmaestre@gmail.com
Bojan Mihaljevic - boki.mihaljevic@gmail.com
"""
from __future__ import division
from numpy import array, shape, where, in1d
import math
import time
import nose

class InformationTheoryTool:
    
    def __init__(self, data):
        """
        """
        # Check if all rows have the same length
        assert (len(data.shape) == 2)
        # Save data
        self.data = data
        self.n_rows = data.shape[0]
        self.n_cols = data.shape[1]
        
        
    def single_entropy(self, x_index, log_base, debug=False):
        """
        Calculate the entropy of a random variable
        """
        # Check if index are into the bounds
        assert (x_index >= 0 and x_index <= self.n_rows)
        # Variable to return entropy
        summation = 0.0
        # Get unique values of random variables
        values_x = set(self.data[x_index])
        # Print debug info
        if debug:
            print('Entropy of')
            print(self.data[x_index])
        # For each random value
        for value_x in values_x:
            px = shape(where(self.data[x_index] == value_x))[1] / self.n_cols
            if px > 0.0:
                summation += px * math.log(px, log_base)
            if debug:
                print(f'({value_x}) px:{px}')
        if summation == 0.0:
            return summation
        else:
            return -summation
        
        
    def entropy(self, x_index, y_index, log_base, debug=False):
        """
        Calculate the entropy between two random variables
        """
        assert (x_index >= 0 and x_index <= self.n_rows)
        assert (y_index >= 0 and y_index <= self.n_rows)
        # Variable to return entropy
        summation = 0.0
        # Get unique values of random variables
        values_x = set(self.data[x_index])
        values_y = set(self.data[y_index])
        # Print debug info
        if debug:
            print('Entropy between')
            print(self.data[x_index])
            print(self.data[y_index])
        # For each random value pair
        for value_x in values_x:
            for value_y in values_y:
                pxy = len(where(in1d(where(self.data[x_index] == value_x)[0], 
                                     where(self.data[y_index] == value_y)[0]) == True)[0]) / self.n_cols
                if pxy > 0.0:
                    summation += pxy * math.log(pxy, log_base)
                if debug:
                    print(f'({value_x},{value_y}) pxy:{pxy}')
        if summation == 0.0:
            return summation
        else:
            return -summation
        
        
    def mutual_information(self, x_index, y_index, log_base, debug=False):
        """
        Calculate and return Mutual Information between two random variables
        """
        # Check if index are into the bounds
        assert (x_index >= 0 and x_index <= self.n_rows)
        assert (y_index >= 0 and y_index <= self.n_rows)
        # Variable to return MI
        summation = 0.0
        # Get unique values of random variables
        values_x = set(self.data[x_index])
        values_y = set(self.data[y_index])
        # Print debug info
        if debug:
            print('MI between')
            print(self.data[x_index])
            print(self.data[y_index])
        # For each random value pair
        for value_x in values_x:
            for value_y in values_y:
                px = shape(where(self.data[x_index] == value_x))[1] / self.n_cols
                py = shape(where(self.data[y_index] == value_y))[1] / self.n_cols
                pxy = len(where(in1d(where(self.data[x_index] == value_x)[0], 
                                     where(self.data[y_index] == value_y)[0]) == True)[0]) / self.n_cols
                if pxy > 0.0:
                    summation += pxy * math.log((pxy / (px * py)), log_base)
                if debug:
                    print(f'({value_x},{value_y}) px:{px} py:{py} pxy:{pxy}')
        return summation


In [1]:
import pandas as pd
import os
from scipy.interpolate import PchipInterpolator
import numpy as np
import copy
from scipy.stats import spearmanr
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
#dataframe datos de compositores 

datos_composers = {}
carpeta = r'Sequences\labels'
archivos_en_carpeta = os.listdir(carpeta)
index0 = 0
indice = 0

for archivo in archivos_en_carpeta:
    ruta_completa = os.path.join(carpeta, archivo)
    serie = pd.read_csv(ruta_completa, header = None)
    composer = archivo.split('-')[1].capitalize() # nombre compositor
    datos_composers[composer] = {} #genero bibio para composer
    datos_composers[composer]['Birth_year'] = archivo.split('-')[0] #año de nacimiento
    index1 = serie.iloc[0, 0].split('\t')[0] #el # del primer serie del composer
    index2 = int(serie.iloc[len(serie)-3, 0].split('\t')[0]) - index0 # # Piezas
    index0 = index2 + index0 # numero total de piezas anteriores
    datos_composers[composer]['# Piezas'] = index2 # Piezas
    datos_composers[composer]['Indice'] = indice
    indice += 1

composers = {}
M = 0
carpeta = r'Sequences\Series'
archivos_en_carpeta = os.listdir(carpeta)

for archivo in archivos_en_carpeta:
    ruta_completa = os.path.join(carpeta, archivo)
    serie = pd.read_csv(ruta_completa)
    # escoge una serie
    composer = archivo.split('-')[1].capitalize() # nombre compositor
    composers[composer] = {}

    for pieza in range( datos_composers[composer]['# Piezas'] ):
        N = serie.iloc[0, 0].split('\t')[1] # # de elementos por pieza
        M = int(N) + M
        index_n1 = 0 
        index_n2 = int(N)+2 
        serie_n = serie[index_n1 + 2:index_n2].reset_index(drop=True) # resetear index
        serie = serie[index_n2 +1:] # recortar serie Original
        serie_n.index += 1 # que index empiece desde 1
        num_serie_T = serie.columns[0]  # numero de serie de todo el dataset
        num_serie = pieza + 1
        composers[composer]['Serie_'+str(num_serie)] = serie_n.squeeze().to_numpy().astype(float) # agregamos pieza al dicc composer con key como # serie

###
###

composers_depurado = copy.deepcopy(composers)
datos_composers_depurado = copy.deepcopy(datos_composers)

for i,composer in enumerate(composers.keys()):
    d = 0
    for pieza in composers[composer].keys():
        if len(composers[composer][pieza])//2 < 400:
            del composers_depurado[composer][pieza]
            d = d + 1
    datos_composers_depurado[composer]['# Piezas'] = datos_composers[composer]['# Piezas'] - d


# 40 promedio de numero de piezas por compositor
composers_depurado_v2 = copy.deepcopy(composers_depurado)
composers_depurado_v2_keychange = copy.deepcopy(composers_depurado_v2)
datos_composers_depurado_v2 = copy.deepcopy(datos_composers_depurado)

for composer in composers.keys():
    if datos_composers_depurado[composer]['# Piezas'] < 30:
        del composers_depurado_v2[composer]
        del datos_composers_depurado_v2[composer]
    
for i,composer in enumerate(composers_depurado_v2.keys()):
    datos_composers_depurado_v2[composer]['Indice'] = i 

for composer in composers_depurado_v2.keys():
    for i,serie in enumerate(composers_depurado_v2[composer].keys()):
        composers_depurado_v2_keychange[composer]['Serie_' + str(i+1)] = composers_depurado_v2_keychange[composer].pop(serie)

print(" # de compositores restantes: ", len(composers_depurado_v2))

 # de compositores restantes:  19


In [3]:
def log2(x):
    x[x == 0] = 1  # Evitar problemas con log(0)
    return np.log2(x)

# Función para calcular la información mutua
def calcular_informacion_mutua(x, y, bins=20):
    # Discretizar ambas variables en bins
    hist_conj, edges_x, edges_y = np.histogram2d(x, y, bins=bins)
    prob_conj = hist_conj / np.sum(hist_conj)  # Probabilidad conjunta

    # Probabilidades marginales
    prob_x = np.sum(prob_conj, axis=1)  # Marginal en X
    prob_y = np.sum(prob_conj, axis=0)  # Marginal en Y

    # Entropías
    H_x = -np.sum(prob_x * log2(prob_x))
    H_y = -np.sum(prob_y * log2(prob_y))
    H_conj = -np.sum(prob_conj * log2(prob_conj))

    # Información mutua
    I_xy = H_x + H_y - H_conj
    return I_xy, H_x, H_y, H_conj

In [11]:
import random
Js = np.load('J_composers_global.npy')
xi = np.load('xi_index_global.npy')
# data2 = np.array([
#     generar_uniforme_centrada(2677, 1e-6),  # Variable X
#     generar_uniforme_centrada(2677, 1e-6)   # Variable Y
data2 = np.array([
    np.array(np.linspace(-200,200,1000)),  # Variable X
    (np.linspace(-200,200,1000)**2)   # Variable Y
])

pts_interp = 15
Js = np.array([])
for composer in composers_depurado_v2.keys():
        array = 1-np.load('J_hermite_sincorte_depurado/interp_'+str(pts_interp)+'/'+ str(datos_composers[composer]['Birth_year']) + '_J_interp_' + str(composer) + '_herm.npy')
        Js = np.concatenate((Js,array))
Js_random = data2[0,:]
null = []
print(calcular_informacion_mutua(Js,xi))
# for _ in range(1000):
#     for i in range(100):
#         random.shuffle(Js_random)
#     null.append(calcular_informacion_mutua(Js_random,data2[1,:])[0])
#     print(len(null))
# MI_index = np.abs(calcular_informacion_mutua(data2[0,:],data2[1,:])[0]- np.mean(null))/np.std(null)
# MI_index

#2 0.04934371108605262
#3 0.04755197883631457
#4 0.054119239172703004
#5 0.057229357687264226
#15 0.05934594566672313
# no mecetas 0.05444613157353828
#3: 0.04698538041088529

# calcular_informacion_mutua(data2[0,:],data2[1,:])

(0.10575395309166691, 3.9597427501269404, 3.0885452854191504, 6.942534082454424)


In [ ]:
import numpy as np

Js = np.load('J_composers_global.npy')
xi = np.load('xi_index_global.npy')
print(xi)
# Define tus datos en forma de array
data = np.array([
    np.load('xi_index_global.npy'),  # Variable X
    np.load('J_composers_lineal_global.npy')   # Variable Y
])
print(np.shape(data))
# np.load('J_composers_lineal_global')
# np.load('J_composers_hermite_global')


# Crear el objeto InformationTheoryTool
it_tool = InformationTheoryTool(data)

# Calcular la información mutua entre X (fila 0) e Y (fila 1)
mi = it_tool.mutual_information(0, 1, 2)  # Base 2 para bits
print(f"Información mutua entre X e Y: {mi:.4f}")


In [10]:
def generar_uniforme_centrada(n, varianza):
    # Calcular el límite superior e inferior de la distribución uniforme
    limite = np.sqrt(varianza)
    # Generar n números aleatorios con distribución uniforme entre -limite y limite
    return np.random.uniform(-limite, limite, n)

In [20]:
import numpy as np
# data2 = np.array([
#     generar_uniforme_centrada(2677, 1e-6),  # Variable X
#     generar_uniforme_centrada(2677, 1e-6)   # Variable Y
# ])
# print(np.shape(data))
# np.load('J_composers_lineal_global')
# np.load('J_composers_hermite_global')
data2 = np.array([
    np.array(np.linspace(-200,200,1000)),  # Variable X
    (np.linspace(-200,200,1000))   # Variable Y
])

# Crear el objeto InformationTheoryTool
# it_tool2 = InformationTheoryTool(data2)

# # Calcular la información mutua entre X (fila 0) e Y (fila 1)
# mi2 = it_tool2.mutual_information(0, 1, 2)  # Base 2 para bits
# print(f"Información mutua entre X e Y: {mi2:.4f}")

In [13]:
import numpy as np

def mutual_information(X, Y):
    """
    Calcula la información mutua discreta entre dos variables aleatorias X e Y.

    Parámetros:
        X (array-like): Variable aleatoria discreta 1.
        Y (array-like): Variable aleatoria discreta 2.

    Retorna:
        float: Información mutua entre X e Y.
    """
    # Asegurarse de que X e Y sean numpy arrays
    X = np.asarray(X)
    Y = np.asarray(Y)
    
    # Obtener los valores únicos y sus conteos para las distribuciones marginales
    unique_x, counts_x = np.unique(X, return_counts=True)
    unique_y, counts_y = np.unique(Y, return_counts=True)
    
    # Calcular las probabilidades marginales
    p_x = counts_x / len(X)
    p_y = counts_y / len(Y)
    
    # Crear una matriz conjunta
    joint_counts = np.zeros((len(unique_x), len(unique_y)))
    
    for i, x_val in enumerate(unique_x):
        for j, y_val in enumerate(unique_y):
            joint_counts[i, j] = np.sum((X == x_val) & (Y == y_val))
    
    # Convertir los conteos conjuntos a probabilidades
    p_xy = joint_counts / len(X)
    
    # Calcular la información mutua
    mi = 0
    for i in range(len(unique_x)):
        for j in range(len(unique_y)):
            if p_xy[i, j] > 0:  # Evitar log(0)
                mi += p_xy[i, j] * np.log(p_xy[i, j] / (p_x[i] * p_y[j]))
    
    return mi


In [ ]:
import numpy as np
import os
from nltk.util import ngrams
from nltk.probability import FreqDist

# Ruta de la carpeta con los archivos
carpeta = r'Contenido'

# Función para calcular log2, evitando problemas con ceros
def log2(x):
    x[x == 0] = 1  # Evita log2(0)
    return np.log2(x)

# Función para discretizar datos continuos en bins
def discretizar_datos(data, bins=10):
    data_discretizada = np.digitize(data, bins=np.linspace(np.min(data), np.max(data), bins))
    return data_discretizada

# Función para calcular información mutua entre dos variables
def calcular_informacion_mutua(data, bins=10):
    # Discretizar los datos continuos
    valores_discretizados = discretizar_datos(data[:, 1], bins=bins)
    
    # Obtener valores únicos y calcular probabilidades marginales
    valores_unicos, conteos = np.unique(valores_discretizados, return_counts=True)
    prob = conteos / np.sum(conteos)
    
    # Calcular bigramas y frecuencias conjuntas
    bigramas = list(ngrams(valores_discretizados, 2))
    frecuencias_bigrama = FreqDist(bigramas)
    
    # Crear matriz de frecuencias conjuntas
    matrix_counts = np.zeros((len(valores_unicos), len(valores_unicos)))
    for bigrama, frecuencia in frecuencias_bigrama.items():
        pos1 = np.where(valores_unicos == bigrama[0])[0][0]
        pos2 = np.where(valores_unicos == bigrama[1])[0][0]
        matrix_counts[pos1, pos2] = frecuencia
    
    # Convertir frecuencias conjuntas a probabilidades conjuntas
    matrix_conj = matrix_counts / len(bigramas)
    
    # Calcular probabilidades marginales
    prob_x = np.sum(matrix_conj, axis=1)
    prob_y = np.sum(matrix_conj, axis=0)
    
    # Calcular entropías marginales y conjuntas
    Ent_x = -np.sum(prob_x * log2(prob_x))
    Ent_y = -np.sum(prob_y * log2(prob_y))
    Ent_conj = -np.sum(matrix_conj * log2(matrix_conj))
    
    # Calcular información mutua
    Inf_mut = Ent_x + Ent_y - Ent_conj
    return Inf_mut

# Leer archivos de la carpeta y calcular información mutua
archivos_en_carpeta = os.listdir(carpeta)
resultados = {}

for archivo in archivos_en_carpeta:
    ruta_completa = os.path.join(carpeta, archivo)
    data = np.genfromtxt(ruta_completa)
    # Calcular información mutua con 10 bins
    mi = calcular_informacion_mutua(data, bins=10)
    resultados[archivo] = mi

# Imprimir resultados
for archivo, mi in resultados.items():
    print(f"La información mutua del archivo {archivo} es: {mi}")


In [ ]:
mutual_information(data2[0,:], data2[1,:])

In [ ]:
Js = np.load('J_composers_global.npy')
xi = np.load('xi_index_global.npy')
mutual_information(Js, xi)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(Js, xi, '.')